<a href="https://colab.research.google.com/github/FaisalWant/kafka/blob/main/kafka.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

setup Kafka:


- Kafka (Brokers: 127.0.0.1:9092)
- Zookeeper (Node: 127.0.0.1:2181)

In [1]:
!curl -sSOL https://dlcdn.apache.org/kafka/3.1.0/kafka_2.13-3.1.0.tgz
!tar -xzf kafka_2.13-3.1.0.tgz

In [2]:
!ls

kafka_2.13-3.1.0  kafka_2.13-3.1.0.tgz	sample_data


Using the default configurations (provided by Apache Kafka) for spinning up the instances.

In [3]:
!./kafka_2.13-3.1.0/bin/zookeeper-server-start.sh -daemon ./kafka_2.13-3.1.0/config/zookeeper.properties
!./kafka_2.13-3.1.0/bin/kafka-server-start.sh -daemon ./kafka_2.13-3.1.0/config/server.properties
!echo "Waiting for 10 secs until kafka and zookeeper services are up and running"
!sleep 10

Waiting for 10 secs until kafka and zookeeper services are up and running


In [ ]:
!ps -ef | grep kafka

In [5]:
!./kafka_2.13-3.1.0/bin/kafka-topics.sh --create --bootstrap-server 127.0.0.1:9092 --replication-factor 1 --partitions 1 --topic susy-train
!./kafka_2.13-3.1.0/bin/kafka-topics.sh --create --bootstrap-server 127.0.0.1:9092 --replication-factor 1 --partitions 2 --topic susy-test


Created topic susy-train.
Created topic susy-test.


In [6]:
!./kafka_2.13-3.1.0/bin/kafka-topics.sh --describe --bootstrap-server 127.0.0.1:9092 --topic susy-train
!./kafka_2.13-3.1.0/bin/kafka-topics.sh --describe --bootstrap-server 127.0.0.1:9092 --topic susy-test

Topic: susy-train	TopicId: Y_MUYfO5SYWIwajBxViMhA	PartitionCount: 1	ReplicationFactor: 1	Configs: segment.bytes=1073741824
	Topic: susy-train	Partition: 0	Leader: 0	Replicas: 0	Isr: 0
Topic: susy-test	TopicId: CgBrPcn9SQuwc8F3BFY_RA	PartitionCount: 2	ReplicationFactor: 1	Configs: segment.bytes=1073741824
	Topic: susy-test	Partition: 0	Leader: 0	Replicas: 0	Isr: 0
	Topic: susy-test	Partition: 1	Leader: 0	Replicas: 0	Isr: 0


In [11]:
!./kafka_2.13-3.1.0/bin/kafka-topics.sh --list --bootstrap-server 127.0.0.1:9092


susy-test
susy-train


In [ ]:
!./kafka_2.13-3.1.0/bin/kafka-console-producer.sh  --broker-list 127.0.0.1:9092  --topic susy-train 

In [ ]:
!./kafka_2.13-3.1.0/bin/kafka-console-consumer.sh --topic susy-train --bootstrap-server localhost:9092 --from-beginning

Using Python to interact with kafka

In [ ]:
!pip install confluent-kafka

In [ ]:
import asyncio 

from confluent_kafka import Consumer, Producer 
from confluent_kafka.admin import AdminClient, NewTopic 

BROKER_URL= "PLAINTEXT://localhost:9092" 
TOPIC_NAME= "susy-train" 

async def produce(topic_name): 
  """ Produce data into kafka topic""" 
  p= Producer({"bootstrap.servers":BROKER_URL})
  curr_iteration=0 
  
  while True: 
    p.produce(TOPIC_NAME, f"Message:{curr_iteration}")
    curr_iteration+=1 


    await asyncio.sleep(1)




async def consume(topic_name): 
  """ consume data from kafka topic""" 
  c= Consumer({"bootstrap.servers":BROKER_URL, "group.id":"First-Python_consumer"})
  c.subscribe([TOPIC_NAME]) 

  while True:
    message= c.poll(1.0)
    if message is None :
      print("No message received") 
    
    elif message.error() is not None: 
      print(f"Message had an error{ message.error()}")
    else: 
      print(f"Key: {message.key()}, Value:{message.value()}")

    await asyncio.sleep(1) 


async def produce_consume():
  """ Runs the producer and consumer task"""
  t1= asyncio.create_task(produce(TOPIC_NAME)) 
  t2= asyncio.create_task(consume(TOPIC_NAME))

  await t1
  await t2



def main():
  client= AdminClient({"bootstrap.servers":BROKER_URL})
  topic= NewTopic(TOPIC_NAME, num_partitions=1, replication_factor=1)

  client.create_topics([topic])

  try: 
    asyncio.run(produce_consume()) 
  
  except: 
    print("Shutting Down") 
  
  finally: 
    client.delete_topics([topic]) 
    pass 



main()

In [ ]:
#!tail -f /var/log/journal/confluent-kafka.service.log    ---Kafka logs